In [4]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import boto3

In [5]:
# FONCTION AFFICHAGE DES 3 DATAFRAMES

def fn_affichage_classements(url):
    response = requests.get(url)
    # return(response)
    if response.status_code == 200:
        # return('test')

        # je crée une variable soup surn laquelle j'applique BeautifulSoup
        # avec en paramètre url.text et je découpe avec html.parser
        soup = BeautifulSoup(response.text, 'html.parser')
        # return(soup)   

        # NOM EQUIPES
        # je recherche les noms de toutes les équipes avec la balise a et la classe affichée sur la page internet
        list_nom_equipes = soup.find_all('a', {'class': 'base-link base-link--black'})

        # je crée une boucle insérer les noms des equipes dans une liste
        list_team = []
        for ele in list_nom_equipes:
                ele = str(ele)
                # Appliquer la regex à chaque élément `ele`
                match = re.search(r'>([^<]+)<', ele)
                if match:  # Vérifier si la regex a trouvé un résultat
                    list_team.append(match.group(1).strip())
        # print(list_team)    
        
        points_equipes = soup.find_all('div', {'class': 'table-line__cell--small'})
        list_points_team = []
        for points in points_equipes:
            # print(points)
            points = str(points)
            regex_points = re.search(r'[0-9]+', points)
            # print(regex_points)
            # regex_points[0]
            # print(regex_points[0])
            list_points_team.append(regex_points[0])

        all_team = soup.find_all('div', {'class': 'table-line table-line--ranking-full table-line--ranking-scrollable'})
        list_matchs_joues = []
        list_matchs_gagnes = []
        list_matchs_nuls = []
        list_matchs_perdus = []

        for team in range(0, 14):
            for data in range(1, 5):
                if data == 1:
                    points_test = all_team[team].find_all('div', {'class': ''})[data]
                    # print(re.search(r'[0-9]+', str(points_test))[0])
                    list_matchs_joues.append(re.search(r'[0-9]+', str(points_test))[0])
                elif data == 2:
                    points_test = all_team[team].find_all('div', {'class': ''})[data]
                    # print(re.search(r'[0-9]+', str(points_test))[0])
                    list_matchs_gagnes.append(re.search(r'[0-9]+', str(points_test))[0])
                elif data == 3:
                    points_test = all_team[team].find_all('div', {'class': ''})[data]
                    # print(re.search(r'[0-9]+', str(points_test))[0])
                    list_matchs_nuls.append(re.search(r'[0-9]+', str(points_test))[0])
                else:
                    points_test = all_team[team].find_all('div', {'class': ''})[data]
                    # print(re.search(r'[0-9]+', str(points_test))[0])
                    list_matchs_perdus.append(re.search(r'[0-9]+', str(points_test))[0])

        return pd.DataFrame({
                "Equipes": list_team,
                "Pts": list_points_team,
                "Matchs joués": list_matchs_joues,
                "Gagnés": list_matchs_gagnes,
                "Nuls": list_matchs_nuls,
                "Perdus": list_matchs_perdus
                })
    else:
        return "Erreur d'url"

df_classement_general = fn_affichage_classements("https://top14.lnr.fr/classement")
df_classement_domicile = fn_affichage_classements("https://top14.lnr.fr/classement/domicile")
df_classement_exterieur = fn_affichage_classements("https://top14.lnr.fr/classement/exterieur")

import os

# Créer un dossier pour stocker les fichiers JSON
os.makedirs("data_brute", exist_ok=True)

# Sauvegarde en JSON
df_classement_general.to_json("data_brute/classement_general.json", orient="records", force_ascii=False, indent=4)
df_classement_domicile.to_json("data_brute/classement_domicile.json", orient="records", force_ascii=False, indent=4)
df_classement_exterieur.to_json("data_brute/classement_exterieur.json", orient="records", force_ascii=False, indent=4)

print("✅ Fichiers JSON créés avec succès !")

for fichier in ["classement_general.json", "classement_domicile.json", "classement_exterieur.json"]:
    chemin = f"data_brute/{fichier}"
    if os.path.exists(chemin):
        print(f"✅ {chemin} est bien créé !")
    else:
        print(f"❌ Erreur : {chemin} est manquant !")

import json

with open("data_brute/classement_general.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(json.dumps(data[:3], indent=4, ensure_ascii=False))  # Affiche les 3 premières lignes




# dico_urls = {"https://top14.lnr.fr/classement": "Classement général", "https://top14.lnr.fr/classement/domicile": "Classement domicile", "https://top14.lnr.fr/classement/exterieur": "Classement extèrieur"}
# for url in dico_urls.keys():
#     dico_urls[url]
#     print(dico_urls[url])
#     # print(url)
#     fn_affichage_classements(url)

✅ Fichiers JSON créés avec succès !
✅ data_brute/classement_general.json est bien créé !
✅ data_brute/classement_domicile.json est bien créé !
✅ data_brute/classement_exterieur.json est bien créé !
[
    {
        "Equipes": "Union Bordeaux-Bègles",
        "Pts": "51",
        "Matchs joués": "15",
        "Gagnés": "11",
        "Nuls": "0",
        "Perdus": "4"
    },
    {
        "Equipes": "Stade Toulousain",
        "Pts": "51",
        "Matchs joués": "15",
        "Gagnés": "10",
        "Nuls": "1",
        "Perdus": "4"
    },
    {
        "Equipes": "RC Toulon",
        "Pts": "48",
        "Matchs joués": "15",
        "Gagnés": "10",
        "Nuls": "0",
        "Perdus": "5"
    }
]


# script d'upload vers S3